In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
# from format import format

def getelementinlist(list,element):
    try:
        return list[element]
    except:
        return '-'

def getfinancialreportingdf(ticker):

    urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
    urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'
    
    text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"lxml")
    text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"lxml")

    # Income statement
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
    epslist = []
    netincomelist = []
    longtermdebtlist = []
    interestexpenselist = []
    ebitdalist = []

    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in title.findNextSiblings (attrs={'class': 'valueCell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

    # Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
    equitylist = []

    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append ([td.text for td in title.findNextSiblings (attrs={'class': 'valueCell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

    eps = getelementinlist(epslist,0) #function(table,row)
    epsgrowth = getelementinlist(epslist,1)
    netincome = getelementinlist(netincomelist,0)
    shareholderequity = getelementinlist(equitylist,0)
    roa = getelementinlist(equitylist,1)

    longtermdebt = getelementinlist(longtermdebtlist,0)
    interestexpense = getelementinlist(interestexpenselist,0)
    ebitda = getelementinlist(ebitdalist,0)

    df = pd.DataFrame({
        'eps': eps,
        'epsgrowth': epsgrowth,
        'netincome': netincome,
        'shareholderequity': shareholderequity,
        'roa': roa,
        'longtermdebt': longtermdebt,
        'interestexpense': interestexpense,
        'ebitda': ebitda
    },
        index=[2015, 2016, 2017, 2018, 2019]
    )

    return df

def stockProfile(ticker):

    urlprofile = "https://www.marketwatch.com/investing/stock/"+ticker+"/profile"
    text_soup_profile = BeautifulSoup(requests.get(urlprofile).text,"lxml")

    # Profile data
    titlesprofile = text_soup_profile.findAll('p', {'class': 'column'})
    pbr = float("nan")

    for title in titlesprofile:
        if 'Price to Book Ratio' in title.text:
            for td in title.findNextSiblings(attrs={'class': 'data lastcolumn'}):
                pbr = float(td.text)
                
    return pbr

# Getting financial reporting df
def getfinancialreportingdfformatted(ticker):
    df = getfinancialreportingdf(ticker)
    # Format all the number in dataframe
    dfformatted = df.apply(format)

    # Adding roe, interest coverage ratio
    dfformatted['roe'] = dfformatted.netincome/dfformatted.shareholderequity
    dfformatted['interestcoverageratio'] = dfformatted.ebitda/dfformatted.interestexpense

#     Insert ticker and df
    return dfformatted

In [ ]:
stock_list = ['tsla', 'amd', 'nio', 'uber']
sp500 = ['a', 'aa', 'aapl', 'abbv', 'abc', 'abt', 'ace', 'aci', 'acn', 'act', 'adbe', 'adi', 'adm', 'adp', 'adsk', 'adt', 'aee', 'aeo', 'aep', 'aes', 'aet', 'afl', 'agn', 'aig', 'aiv', 'aiz', 'akam', 'all', 'altr', 'alxn', 'amat', 'amd', 'amgn', 'amp', 'amt', 'amzn', 'an', 'anf', 'ann', 'aon', 'apa', 'apc', 'apd', 'aph', 'apol', 'arg', 'arna', 'aro', 'ati', 'atvi', 'avb', 'avp', 'avy', 'axp', 'azo', 'ba', 'bac', 'bax', 'bbby', 'bbry', 'bbt', 'bby', 'bcr', 'bdx', 'beam', 'ben', 'bf-b', 'bhi', 'big', 'biib', 'bk', 'bks', 'blk', 'bll', 'bmc', 'bms', 'bmy', 'brcm', 'brk-b', 'bsx', 'btu', 'bwa', 'bxp', 'c', 'ca', 'cab', 'cag', 'cah', 'cam', 'cat', 'cb', 'cbg', 'cbs', 'cce', 'cci', 'ccl', 'celg', 'cern', 'cf', 'cfn', 'chk', 'chrw', 'ci', 'cim', 'cinf', 'cl', 'clf', 'clx', 'cma', 'cmcsa', 'cme', 'cmg', 'cmi', 'cms', 'cnp', 'cnx', 'cof', 'cog', 'coh', 'col', 'cop', 'cost', 'cov', 'cpb', 'crm', 'csc', 'csco', 'csx', 'ctas', 'ctl', 'ctsh', 'ctxs', 'cvc', 'cvs', 'cvx', 'd', 'dal', 'dd', 'dds', 'de', 'dell', 'df', 'dfs', 'dg', 'dgx', 'dhi', 'dhr', 'dis', 'disca', 'dks', 'dlph', 'dltr', 'dlx', 'dnb', 'dnr', 'do', 'dov', 'dow', 'dps', 'dri', 'dsw', 'dte', 'dtv', 'duk', 'dva', 'dvn', 'ea', 'ebay', 'ecl', 'ed', 'efx', 'eix', 'el', 'emc', 'emn', 'emr', 'eog', 'eqr', 'eqt', 'esrx', 'esv', 'etfc', 'etn', 'etr', 'ew', 'exc', 'expd', 'expe', 'expr', 'f', 'fast', 'fb', 'fcx', 'fdo', 'fdx', 'fe', 'ffiv', 'fhn', 'fis', 'fisv', 'fitb', 'fl', 'flir', 'flr', 'fls', 'flws', 'fmc', 'fosl', 'frx', 'fslr', 'fti', 'ftr', 'gas', 'gci', 'gd', 'ge', 'ges', 'gild', 'gis', 'glw', 'gm', 'gmcr', 'gme', 'gnw', 'goog', 'gpc', 'gps', 'grmn', 'grpn', 'gs', 'gt', 'gww', 'hal', 'har', 'has', 'hban', 'hcbk', 'hcn', 'hcp', 'hd', 'hes', 'hig', 'hog', 'hon', 'hot', 'hov', 'hp', 'hpq', 'hrb', 'hrl', 'hrs', 'hsp', 'hst', 'hsy', 'hum', 'ibm', 'ice', 'iff', 'igt', 'intc', 'intu', 'ip', 'ipg', 'ir', 'irm', 'isrg', 'itw', 'ivz', 'jbl', 'jci', 'jcp', 'jdsu', 'jec', 'jnj', 'jnpr', 'josb', 'joy', 'jpm', 'jwn', 'k', 'key', 'kim', 'klac', 'kmb', 'kmi', 'kmx', 'ko', 'kr', 'krft', 'kss', 'ksu', 'l', 'leg', 'len', 'lh', 'life', 'lll', 'lltc', 'lly', 'lm', 'lmt', 'lnc', 'lo', 'low', 'lrcx', 'lsi', 'ltd', 'luk', 'luv', 'lyb', 'm', 'ma', 'mac', 'mar', 'mas', 'mat', 'mcd', 'mchp', 'mck', 'mco', 'mcp', 'mdlz', 'mdt', 'met', 'mgm', 'mhfi', 'mjn', 'mkc', 'mmc', 'mmm', 'mnst', 'mo', 'molx', 'mon', 'mos', 'mpc', 'mrk', 'mro', 'ms', 'msft', 'msi', 'mtb', 'mu', 'mur', 'mwv', 'myl', 'nbl', 'nbr', 'ndaq', 'ne', 'nee', 'nem', 'nflx', 'nfx', 'ni', 'nile', 'nke', 'nly', 'noc', 'nok', 'nov', 'nrg', 'nsc', 'ntap', 'ntri', 'ntrs', 'nu', 'nue', 'nvda', 'nwl', 'nwsa', 'nyx', 'oi', 'oke', 'omc', 'orcl', 'orly', 'oxy', 'p', 'payx', 'pbct', 'pbi', 'pcar', 'pcg', 'pcl', 'pcln', 'pcp', 'pdco', 'peg', 'pep', 'petm', 'pets', 'pfe', 'pfg', 'pg', 'pgr', 'ph', 'phm', 'pki', 'pld', 'pll', 'pm', 'pnc', 'pnr', 'pnw', 'pom', 'ppg', 'ppl', 'prgo', 'pru', 'psa', 'psx', 'pwr', 'px', 'pxd', 'qcom', 'qep', 'r', 'rai', 'rdc', 'rf', 'rhi', 'rht', 'rl', 'rok', 'rop', 'rost', 'rrc', 'rsg', 'rsh', 'rtn', 's', 'sai', 'sbux', 'scg', 'schl', 'schw', 'sd', 'se', 'see', 'sfly', 'shld', 'shw', 'sial', 'siri', 'sjm', 'sks', 'slb', 'slm', 'sna', 'sndk', 'sne', 'sni', 'so', 'spg', 'spls', 'srcl', 'sre', 'sti', 'stj', 'stt', 'stx', 'stz', 'swk', 'swn', 'swy', 'syk', 'symc', 'syy', 't', 'tap', 'tdc', 'te', 'teg', 'tel', 'ter', 'tgt', 'thc', 'tibx', 'tif', 'tjx', 'tm', 'tmk', 'tmo', 'trip', 'trow', 'trv', 'tsla', 'tsn', 'tso', 'tss', 'twc', 'twx', 'txn', 'txt', 'tyc', 'ua', 'unh', 'unm', 'unp', 'ups', 'urbn', 'usb', 'utx', 'v', 'vale', 'var', 'vfc', 'viab', 'vitc', 'vlo', 'vmc', 'vno', 'vprt', 'vrsn', 'vtr', 'vz', 'wag', 'wat', 'wdc', 'wec', 'wfc', 'wfm', 'whr', 'win', 'wlp', 'wm', 'wmb', 'wmt', 'wpo', 'wpx', 'wtw', 'wu', 'wy', 'wyn', 'wynn', 'x', 'xel', 'xl', 'xlnx', 'xom', 'xray', 'xrx', 'xyl', 'yhoo', 'yum', 'zion', 'zlc', 'zmh', 'znga', 'camp', 'cldx', 'ecyt', 'gtn', 'htz', 'nus', 'pvtb', 'qdel', 'snts', 'wgo', 'wwww']

for ticker in sp500:
    stock_financials = getfinancialreportingdf('tsla')
    price_to_book = stockProfile(ticker)
    if price_to_book < .70:
        print(f"{ticker}")
        print(f"price to book ratio: ", price_to_book)
        print(f"financial data for {ticker}:\n", stock_financials)
        print("-" * 80)
    else:
        continue

In [11]:
url = f'https://finance.yahoo.com/quote/UBER/key-statistics'
soup = BeautifulSoup(requests.get(url).text,"lxml")
titlestats = soup.findAll('td', {'class': "Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px)"})
peg = float("nan")

for title in titlestats:
    title_text = title.text
    print(title)

<td class="Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px)" data-reactid="22"><span data-reactid="23">Enterprise Value</span><!-- react-text: 24 --> <!-- /react-text --><!-- react-text: 25 --><!-- /react-text --><sup aria-label="Data derived from multiple sources or calculated by Yahoo Finance." data-reactid="26">3</sup></td>
<td class="Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px)" data-reactid="29"><span data-reactid="30">Trailing P/E</span><!-- react-text: 31 --> <!-- /react-text --><!-- react-text: 32 --><!-- /react-text --><sup aria-label="" data-reactid="33"></sup></td>
<td class="Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px)" data-reactid="37"><span data-reactid="38">Forward P/E</span><!-- react-text: 39 --> <!-- /react-text --><!-- react-text: 40 --><!-- /react-text --><sup aria-label="Data provided by Thomson Reuters." data-reactid="41">1</sup></td>
<td class="Pos(st) Start(0) Bgc($lv2BgColor)

In [12]:
for title in titlestats:
    title_text = title.text
    print(title_text)

Enterprise Value 3
Trailing P/E 
Forward P/E 1
PEG Ratio (5 yr expected) 1
Price/Sales (ttm)
Price/Book (mrq)
Enterprise Value/Revenue 3
Enterprise Value/EBITDA 6
52-Week Change 3
S&P500 52-Week Change 3
52 Week High 3
52 Week Low 3
50-Day Moving Average 3
200-Day Moving Average 3
Avg Vol (10 day) 3
Shares Outstanding 5
Float 
% Held by Insiders 1
% Held by Institutions 1
Shares Short (Jan 30, 2020) 4
Short Ratio (Jan 30, 2020) 4
Short % of Float (Jan 30, 2020) 4
Short % of Shares Outstanding (Jan 30, 2020) 4
Shares Short (prior month Dec 30, 2019) 4
Forward Annual Dividend Yield 4
Trailing Annual Dividend Rate 3
Trailing Annual Dividend Yield 3
5 Year Average Dividend Yield 4
Payout Ratio 4
Dividend Date 3
Ex-Dividend Date 4
Last Split Factor 2
Last Split Date 3
Most Recent Quarter (mrq)
Operating Margin (ttm)
Return on Equity (ttm)
Revenue Per Share (ttm)
Quarterly Revenue Growth (yoy)
Gross Profit (ttm)
EBITDA 
Net Income Avi to Common (ttm)
Diluted EPS (ttm)
Quarterly Earnings Grow

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

def yahooStockStats(ticker):
    url = f'https://finance.yahoo.com/quote/{ticker}/key-statistics'
    soup = BeautifulSoup(requests.get(url).text,"lxml")
    titlestats = soup.findAll('td', {'class': "Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px)"})
    
    
    pbr = float("nan")
    peg5 = float("nan")
    
    for title in titlestats:
        title_text = title.text
        
        if title_text.startswith('Trailing P/E'):
            per_lst = title.findNextSiblings(attrs={'class': "Fz(s) Fw(500) Ta(end) Pstart(10px) Miw(60px)"})
            try:
                per = float(per_lst[0].text)
            except:
                per = float("nan")
        elif title_text.startswith('Price/Book (mrq)'):
            pbr_lst = title.findNextSiblings(attrs={'class': "Fz(s) Fw(500) Ta(end) Pstart(10px) Miw(60px)"})
            try:
                pbr = float(pbr_lst[0].text)
            except:
                pbr = float("nan")
        elif title_text.startswith('PEG Ratio (5 yr expected)'):
            peg5_lst = title.findNextSiblings(attrs={'class': "Fz(s) Fw(500) Ta(end) Pstart(10px) Miw(60px)"})
            try:
                peg5 = float(peg5_lst[0].text)
            except:
                peg5 = float("nan")
    
    return per, pbr, peg5

In [6]:
# getting russel 3000
tickers = []

def parseRus():
    try:
        readFile = open('russell3000.txt','r').read()
        splitFile = readFile.split('\n')
        for eachLine in splitFile:
            splitLine = eachLine.split(',')
            ticker = splitLine[-1]
            tickers.append(ticker)
            
        print(tickers)
        
    except Exception:
        print('failed in the main loop')
        
parseRus()

['FLWS', 'FCCY', 'SRCE', 'XXII', 'TWOU', 'DDD', 'MMM', 'EGHT', 'ATEN', 'AAON', 'AIR', 'AAN', 'ABT', 'ABBV', 'ABEO', 'ANF', 'ABMD', 'ABM', 'AXAS', 'ACIA', 'ACTG', 'ACHC', 'ACAD', 'AKR', 'AXDX', 'XLRN', 'ACN', 'ACCO', 'ARAY', 'ACRX', 'ACER', 'ACHN', 'ACIW', 'ACRS', 'ACNB', 'ACOR', 'ATVI', 'ATU', 'AYI', 'GOLF', 'ADMS', 'ADUS', 'IOTS', 'ADNT', 'ADMA', 'ADBE', 'ADT', 'ATGE', 'AA', 'ALDR', 'ALDX', 'ALEC', 'ALEX', 'ALX', 'ARE', 'ALXN', 'ALCO', 'ALGN', 'ALKS', 'ALLK', 'Y', 'ATI', 'ABTX', 'ALGT', 'ALLE', 'AGN', 'ALE', 'ADS', 'LNT', 'AMOT', 'ALSN', 'ALLO', 'MDRX', 'ALL', 'ALLY', 'ALNY', 'AOSL', 'GOOGL', 'GOOG', 'ATEC', 'ALTR', 'AYX', 'ATUS', 'ASPS', 'AIMC', 'MO', 'ALTM', 'AMAG', 'AMAL', 'AMZN', 'AMBC', 'AMBA', 'AMBR', 'AMC', 'AMCX', 'DOX', 'AMED', 'AVXL', 'ANDE', 'ANGO', 'ANIP', 'ANIK', 'AXE', 'NLY', 'ANSS', 'ATRS', 'AM', 'AR', 'ANTM', 'ANH', 'AON', 'APA', 'AIV', 'APLS', 'APY', 'APOG', 'ARI', 'AMEH', 'APPF', 'APPN', 'APLE', 'AAPL', 'AIT', 'AMAT', 'AAOI', 'ATR', 'APTV', 'APYX', 'WTR', 'AQ', 'WAAS

In [2]:
stock_list = ['tsla', 'amd', 'nio', 'uber']

for ticker in stock_list:
    try:
        per, pbr, peg5 = yahooStockStats(ticker)
        print("-"*12 + f" {ticker} " + "-"*12)
        print("Trailing P/E: ", per)
        print(f"Price/Book ratio: ", pbr)
        print(f"PEG ratio: ", peg5)
        print("-" * 30)
    except:
        print(f"no info for: {ticker}")

------------ tsla ------------
Trailing P/E:  nan
Price/Book ratio:  24.64
PEG ratio:  3.04
------------------------------
------------ amd ------------
Trailing P/E:  177.6
Price/Book ratio:  22.05
PEG ratio:  1.39
------------------------------
------------ nio ------------
Trailing P/E:  nan
Price/Book ratio:  0.67
PEG ratio:  -0.1
------------------------------
------------ uber ------------
Trailing P/E:  nan
Price/Book ratio:  4.93
PEG ratio:  0.11
------------------------------


In [ ]:
stock_list = ['tsla', 'amd', 'nio', 'uber']
sp500 = ['a', 'aa', 'aapl', 'abbv', 'abc', 'abt', 'ace', 'aci', 'acn', 'act', 'adbe', 'adi', 'adm', 'adp', 'adsk', 'adt', 'aee', 'aeo', 'aep', 'aes', 'aet', 'afl', 'agn', 'aig', 'aiv', 'aiz', 'akam', 'all', 'altr', 'alxn', 'amat', 'amd', 'amgn', 'amp', 'amt', 'amzn', 'an', 'anf', 'ann', 'aon', 'apa', 'apc', 'apd', 'aph', 'apol', 'arg', 'arna', 'aro', 'ati', 'atvi', 'avb', 'avp', 'avy', 'axp', 'azo', 'ba', 'bac', 'bax', 'bbby', 'bbry', 'bbt', 'bby', 'bcr', 'bdx', 'beam', 'ben', 'bf-b', 'bhi', 'big', 'biib', 'bk', 'bks', 'blk', 'bll', 'bmc', 'bms', 'bmy', 'brcm', 'brk-b', 'bsx', 'btu', 'bwa', 'bxp', 'c', 'ca', 'cab', 'cag', 'cah', 'cam', 'cat', 'cb', 'cbg', 'cbs', 'cce', 'cci', 'ccl', 'celg', 'cern', 'cf', 'cfn', 'chk', 'chrw', 'ci', 'cim', 'cinf', 'cl', 'clf', 'clx', 'cma', 'cmcsa', 'cme', 'cmg', 'cmi', 'cms', 'cnp', 'cnx', 'cof', 'cog', 'coh', 'col', 'cop', 'cost', 'cov', 'cpb', 'crm', 'csc', 'csco', 'csx', 'ctas', 'ctl', 'ctsh', 'ctxs', 'cvc', 'cvs', 'cvx', 'd', 'dal', 'dd', 'dds', 'de', 'dell', 'df', 'dfs', 'dg', 'dgx', 'dhi', 'dhr', 'dis', 'disca', 'dks', 'dlph', 'dltr', 'dlx', 'dnb', 'dnr', 'do', 'dov', 'dow', 'dps', 'dri', 'dsw', 'dte', 'dtv', 'duk', 'dva', 'dvn', 'ea', 'ebay', 'ecl', 'ed', 'efx', 'eix', 'el', 'emc', 'emn', 'emr', 'eog', 'eqr', 'eqt', 'esrx', 'esv', 'etfc', 'etn', 'etr', 'ew', 'exc', 'expd', 'expe', 'expr', 'f', 'fast', 'fb', 'fcx', 'fdo', 'fdx', 'fe', 'ffiv', 'fhn', 'fis', 'fisv', 'fitb', 'fl', 'flir', 'flr', 'fls', 'flws', 'fmc', 'fosl', 'frx', 'fslr', 'fti', 'ftr', 'gas', 'gci', 'gd', 'ge', 'ges', 'gild', 'gis', 'glw', 'gm', 'gmcr', 'gme', 'gnw', 'goog', 'gpc', 'gps', 'grmn', 'grpn', 'gs', 'gt', 'gww', 'hal', 'har', 'has', 'hban', 'hcbk', 'hcn', 'hcp', 'hd', 'hes', 'hig', 'hog', 'hon', 'hot', 'hov', 'hp', 'hpq', 'hrb', 'hrl', 'hrs', 'hsp', 'hst', 'hsy', 'hum', 'ibm', 'ice', 'iff', 'igt', 'intc', 'intu', 'ip', 'ipg', 'ir', 'irm', 'isrg', 'itw', 'ivz', 'jbl', 'jci', 'jcp', 'jdsu', 'jec', 'jnj', 'jnpr', 'josb', 'joy', 'jpm', 'jwn', 'k', 'key', 'kim', 'klac', 'kmb', 'kmi', 'kmx', 'ko', 'kr', 'krft', 'kss', 'ksu', 'l', 'leg', 'len', 'lh', 'life', 'lll', 'lltc', 'lly', 'lm', 'lmt', 'lnc', 'lo', 'low', 'lrcx', 'lsi', 'ltd', 'luk', 'luv', 'lyb', 'm', 'ma', 'mac', 'mar', 'mas', 'mat', 'mcd', 'mchp', 'mck', 'mco', 'mcp', 'mdlz', 'mdt', 'met', 'mgm', 'mhfi', 'mjn', 'mkc', 'mmc', 'mmm', 'mnst', 'mo', 'molx', 'mon', 'mos', 'mpc', 'mrk', 'mro', 'ms', 'msft', 'msi', 'mtb', 'mu', 'mur', 'mwv', 'myl', 'nbl', 'nbr', 'ndaq', 'ne', 'nee', 'nem', 'nflx', 'nfx', 'ni', 'nile', 'nke', 'nly', 'noc', 'nok', 'nov', 'nrg', 'nsc', 'ntap', 'ntri', 'ntrs', 'nu', 'nue', 'nvda', 'nwl', 'nwsa', 'nyx', 'oi', 'oke', 'omc', 'orcl', 'orly', 'oxy', 'p', 'payx', 'pbct', 'pbi', 'pcar', 'pcg', 'pcl', 'pcln', 'pcp', 'pdco', 'peg', 'pep', 'petm', 'pets', 'pfe', 'pfg', 'pg', 'pgr', 'ph', 'phm', 'pki', 'pld', 'pll', 'pm', 'pnc', 'pnr', 'pnw', 'pom', 'ppg', 'ppl', 'prgo', 'pru', 'psa', 'psx', 'pwr', 'px', 'pxd', 'qcom', 'qep', 'r', 'rai', 'rdc', 'rf', 'rhi', 'rht', 'rl', 'rok', 'rop', 'rost', 'rrc', 'rsg', 'rsh', 'rtn', 's', 'sai', 'sbux', 'scg', 'schl', 'schw', 'sd', 'se', 'see', 'sfly', 'shld', 'shw', 'sial', 'siri', 'sjm', 'sks', 'slb', 'slm', 'sna', 'sndk', 'sne', 'sni', 'so', 'spg', 'spls', 'srcl', 'sre', 'sti', 'stj', 'stt', 'stx', 'stz', 'swk', 'swn', 'swy', 'syk', 'symc', 'syy', 't', 'tap', 'tdc', 'te', 'teg', 'tel', 'ter', 'tgt', 'thc', 'tibx', 'tif', 'tjx', 'tm', 'tmk', 'tmo', 'trip', 'trow', 'trv', 'tsla', 'tsn', 'tso', 'tss', 'twc', 'twx', 'txn', 'txt', 'tyc', 'ua', 'unh', 'unm', 'unp', 'ups', 'urbn', 'usb', 'utx', 'v', 'vale', 'var', 'vfc', 'viab', 'vitc', 'vlo', 'vmc', 'vno', 'vprt', 'vrsn', 'vtr', 'vz', 'wag', 'wat', 'wdc', 'wec', 'wfc', 'wfm', 'whr', 'win', 'wlp', 'wm', 'wmb', 'wmt', 'wpo', 'wpx', 'wtw', 'wu', 'wy', 'wyn', 'wynn', 'x', 'xel', 'xl', 'xlnx', 'xom', 'xray', 'xrx', 'xyl', 'yhoo', 'yum', 'zion', 'zlc', 'zmh', 'znga', 'camp', 'cldx', 'ecyt', 'gtn', 'htz', 'nus', 'pvtb', 'qdel', 'snts', 'wgo', 'wwww']

for ticker in tickers:
    try:
        per, pbr, peg5 = yahooStockStats(ticker)
        
        if pbr < .70:
            if 0 < peg5 < 1:
                print("-"*12 + f" {ticker} " + "-"*12)
                print("Trailing P/E: ", per)
                print(f"Price/Book ratio: ", pbr)
                print(f"PEG ratio: ", peg5)
                print("-" * 30)
    except:
        print(f"no info for: {ticker}")

no info for: ATU
no info for: ALDR
------------ AMAG ------------
Trailing P/E:  nan
Price/Book ratio:  0.67
PEG ratio:  0.1
------------------------------
------------ AMBC ------------
Trailing P/E:  nan
Price/Book ratio:  0.63
PEG ratio:  0.95
------------------------------
no info for: AMBR
no info for: AQ
no info for: PETX
------------ ASC ------------
Trailing P/E:  nan
Price/Book ratio:  0.58
PEG ratio:  0.15
------------------------------
no info for: AVDR
no info for: BHGE
no info for: BTX
------------ BCEI ------------
Trailing P/E:  2.2
Price/Book ratio:  0.41
PEG ratio:  0.2
------------------------------
------------ CPE ------------
Trailing P/E:  2.78
Price/Book ratio:  0.26
PEG ratio:  0.31
------------------------------
no info for: CBLK
no info for: CHFC
------------ CHK ------------
Trailing P/E:  nan
Price/Book ratio:  0.28
PEG ratio:  0.02
------------------------------
no info for: CHSP
------------ CISN ------------
Trailing P/E:  -0.0
Price/Book ratio:  0.0
PEG 